# HSR Time To Prediction Evaluation

The code for this file is mostly the same as the `Time To Prediciton Evaluation.ipynb`. This file contains the cells and code for evaluating the HSR dataset, which was large and needed to be broken down for effiency.

## Importing Datasets

In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from keras.models import load_model
from keras import metrics
from matplotlib import pyplot
# from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import random
from datetime import datetime
from datetime import timedelta
import os
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_confusion_matrix
# %matplotlib notebook
import tensorflow as tf
from tensorflow.python.client import device_lib
import pprint
from scipy import interpolate

2023-06-26 22:50:54.816358: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 22:50:55.275122: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-26 22:50:55.280399: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-26 22:50:57.688294: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Adding a raise exception for clean execution exit from a cell
class StopExecution(Exception):
    def _render_traceback_(self):
        pass

## Loading up the model and setting up functions

Load model according to window size

Uncomment the `model_url` line and the `wget` line in order to get the necessary model.

In [3]:
WINDOW_SIZE = 6

HSR_datasets = ["HSR_Extended_4_0"]

# model_url = "https://anonymous.4open.science/api/repo/EdgeCatBSPDataset-1840/file/models/model_WS" + str(WINDOW_SIZE) + ".h5"
# print(model_url)

# !wget {model_url} -O "model.h5"

model_eval = load_model("/home/mango/Projects/EDGECAT/CONEXT_EDGECAT_LOCAL/model.h5")
window_size = WINDOW_SIZE

2023-06-26 22:51:07.420350: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-26 22:51:07.425130: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-26 22:51:07.427861: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [4]:
# HELPER FUNCTION DECLARATIONS

# Helper function to convert string to timestamp
def string_to_datetime_incp(timestamp):
  return float(timestamp)


# Helper function to convert strings into datetime objects
def string_to_datetime(timestamp_str):
    try:
        time_b = datetime.strptime(timestamp_str, "%Y-%m-%d %X.%f")
    except:
        time_b = datetime.strptime(timestamp_str, "%Y-%m-%d %X")
    return time_b


# This function extracts the rsrp, rsrq and labels, along with assigning the specific dataframe a "time to prediction"
def add_to_df_wts(filepath, window_size):
  df = pd.read_csv(filepath)
  # df.drop(["Unnamed: 0"], axis=1, inplace=True)

  entries = []

  # Getting timestamp for handover (or at least as close to it as possible)
  last_entry = df.iloc[-1]
  last_entry_ts = last_entry["timestamp"]
  ho_ts_datetime = string_to_datetime(last_entry_ts)

  if df.shape[0] >= window_size:
    for i in range(0, df.shape[0]-window_size):
      # Extracting the last timestamp for this window
      last_window_entry = df.iloc[i+window_size]
      last_window_ts = last_window_entry["timestamp"]
      lw_ts_dt = string_to_datetime(last_window_ts) # last window timestamp datetime object

      # Calculating time to handover and adding it to ttp
      tth = ho_ts_datetime - lw_ts_dt # time to handover
      tth_seconds = tth.total_seconds()

      # Extracting the rest
      temp = df.iloc[i:i+window_size]
      rsrp = temp["rsrp"]
      rsrq = temp["rsrq"]
      rsrp_raw = temp["rsrp_raw"]
      rsrq_raw = temp["rsrq_raw"]
      label = df.iloc[i]
      label = label["label"]

      entry = []
      entry.append(rsrp)
      entry.append(rsrq)
      entry.append(label)
      entry.append(tth_seconds)
      entry.append(rsrp_raw)
      entry.append(rsrq_raw)

      entries.append(entry)

    return entries

  return


# This function extracts the rsrp, rsrq and labels, along with assigning the specific dataframe a "time to prediction"
def add_to_df_wts_incp(filepath, window_size):
  df = pd.read_csv(filepath)
  # df.drop(["Unnamed: 0"], axis=1, inplace=True)

  entries = []

  # Getting timestamp for handover (or at least as close to it as possible)
  last_entry = df.iloc[-1]
  last_entry_ts = last_entry["timestamp"]
  ho_ts_datetime = string_to_datetime_incp(last_entry_ts)

  if df.shape[0] >= window_size:
    for i in range(0, df.shape[0]-window_size):
      # Extracting the last timestamp for this window
      last_window_entry = df.iloc[i+window_size]
      last_window_ts = last_window_entry["timestamp"]
      lw_ts_dt = string_to_datetime_incp(last_window_ts) # last window timestamp datetime object

      # Calculating time to handover and adding it to ttp
      tth = ho_ts_datetime - lw_ts_dt # time to handover
      # tth_seconds = tth.total_seconds()
      tth_seconds = tth

      # Extracting the rest
      temp = df.iloc[i:i+window_size]
      rsrp = temp["rsrp"]
      rsrq = temp["rsrq"]
      rsrp_raw = temp["rsrp_raw"]
      rsrq_raw = temp["rsrq_raw"]
      label = df.iloc[i]
      label = label["label"]

      entry = []
      entry.append(rsrp)
      entry.append(rsrq)
      entry.append(label)
      entry.append(tth_seconds)
      entry.append(rsrp_raw)
      entry.append(rsrq_raw)

      entries.append(entry)

    return entries

  return

# Function to detect if there are no "target" entries in here
def check_valid(sequence_dict, key):
  ho_event_list = sequence_dict[key]
  for i, base_station_entries in enumerate(ho_event_list):
    # each base station contains a sequence of measurements at that point for this specific base station
    if base_station_entries != [] and base_station_entries is not None:
      label = sequence_dict[key][i][0][2]
      # print(sequence_dict[key][i][0][2])
      if label == 1:
        return True
  return False

# Helper function to check if the current handover event is valid to be used for evaluation
def check_valid_2(sd, key):
  ho_event_list = sd[key]
  for i, base_station_entries in enumerate(ho_event_list):
    # each base station contains a sequence of measurements at that point for this specific base station
    if base_station_entries != [] and base_station_entries is not None:
      label = sd[key][i][0][2]
      if label == 1:
        return True
  return False


In [5]:
# Helper function for loading up dataset:
def load_dataset(datasets_list, WINDOW_SIZE):
  # HSR_datasets is a placeholder variable.
  sequence_dict = {}

  ho_event_no = 0 # This counter keeps track of which handover event we have reached
  for master_f in datasets_list:
    dataset_folders = os.listdir(master_f)
    for folder in dataset_folders:
      folder_path = master_f + "/" + folder
      files = os.listdir(folder_path)
      ho_event_no += 1
      curr_hoevent_entries = [] # list containing all the data for the current handover event
      for f in files:
        filepath = folder_path + "/" + f
        entries = add_to_df_wts(filepath, window_size)
        curr_hoevent_entries.append(entries)
      sequence_dict[ho_event_no] = curr_hoevent_entries

  sequence_dict_orig = sequence_dict.copy()

  for key in sequence_dict.keys():
    ho_event_list = sequence_dict[key]
    for i1, entry in enumerate(ho_event_list):
      if entry is not None:
        for i2, subentry in enumerate(entry):
          if subentry[2] == "not_target":
            sequence_dict[key][i1][i2][2] = 0
          else:
            sequence_dict[key][i1][i2][2] = 1

  sequence_dict_renamed = sequence_dict.copy()

  sequence_dict = sequence_dict_orig.copy()

  # Using minmax scaling to bring our features between 0 and 1
  scaler = MinMaxScaler(feature_range=(0,1))

  rsrp_list = []
  rsrq_list = []
  rsrp_raws_list = []
  rsrq_raws_list = []
  labels = []

  for key in sequence_dict.keys():
    ho_event_list = sequence_dict[key]
    if check_valid(sequence_dict, key):
      for i1, entry in enumerate(ho_event_list):
        if entry is not None:
          for i2, subentry in enumerate(entry):
            req_idx = len(rsrp_list)
            rsrp = np.array(sequence_dict[key][i1][i2][0])
            rsrq = np.array(sequence_dict[key][i1][i2][1])
            label = np.array(sequence_dict[key][i1][i2][2])
            rsrp_raws = np.array(sequence_dict[key][i1][i2][4])
            rsrq_raws = np.array(sequence_dict[key][i1][i2][5])
            rsrp_list.append(rsrp)
            rsrq_list.append(rsrq)
            rsrp_raws_list.append(rsrp_raws)
            rsrq_raws_list.append(rsrq_raws)
            labels.append(label)
            sequence_dict[key][i1][i2][0] = req_idx
            sequence_dict[key][i1][i2][1] = req_idx
            sequence_dict[key][i1][i2][4] = req_idx
            sequence_dict[key][i1][i2][5] = req_idx
            # sequence_dict[key][i1][i2][2] = req_idx

  rsrp_tts = scaler.fit_transform(rsrp_list)
  rsrq_tts = scaler.fit_transform(rsrq_list)
  rsrp_raws_tts = scaler.fit_transform(rsrp_raws_list)
  rsrq_raws_tts = scaler.fit_transform(rsrq_raws_list)

  # Combining features
  HSR_features = np.stack([rsrp_tts, rsrq_tts, rsrp_raws_tts, rsrq_raws_tts], axis=2)

  # Reshaping label data
  HSR_labels = np.reshape(labels, (len(labels),1))

  return HSR_features, HSR_labels, sequence_dict, sequence_dict_orig

# Helper function specifically for loading the INCP dataset, which has a slightly different format (both in terms of data types and column titles)
def load_incp_dataset(WINDOW_SIZE):
  # Iterate over the files and populate sequence_dict
  dataset_master_folders = ["icnp-dataset/ATT", "icnp-dataset/TMobile"]

  sequence_dict = {}

  ho_event_no = 0 # This counter keeps track of which handover event we have reached
  for master_f in dataset_master_folders:
    dataset_folders = os.listdir(master_f)
    for folder in dataset_folders:
      folder_path = master_f + "/" + folder
      files = os.listdir(folder_path)
      ho_event_no += 1
      curr_hoevent_entries = [] # list containing all the data for the current handover event
      for f in files:
        filepath = folder_path + "/" + f
        entries = add_to_df_wts_incp(filepath, WINDOW_SIZE)
        curr_hoevent_entries.append(entries)
      sequence_dict[ho_event_no] = curr_hoevent_entries

  sequence_dict_orig = sequence_dict.copy()

  # Renaming the labels and converting into required data types

  # sequence_dict = sequence_dict_orig.copy()

  for key in sequence_dict.keys():
    ho_event_list = sequence_dict[key]
    for i1, entry in enumerate(ho_event_list):
      if entry is not None:
        for i2, subentry in enumerate(entry):
          if subentry[2] == "not_target":
            sequence_dict[key][i1][i2][2] = 0
          else:
            sequence_dict[key][i1][i2][2] = 1

  sequence_dict_renamed = sequence_dict.copy()

  sequence_dict = sequence_dict_orig.copy()

  # Using minmax scaling to bring our features between 0 and 1
  scaler = MinMaxScaler(feature_range=(0,1))

  rsrp_list = []
  rsrq_list = []
  rsrp_raws_list = []
  rsrq_raws_list = []
  labels = []

  for key in sequence_dict.keys():
    ho_event_list = sequence_dict[key]
    if check_valid(sequence_dict, key):
      for i1, entry in enumerate(ho_event_list):
        if entry is not None:
          for i2, subentry in enumerate(entry):
            req_idx = len(rsrp_list)
            rsrp = np.array(sequence_dict[key][i1][i2][0])
            rsrq = np.array(sequence_dict[key][i1][i2][1])
            label = np.array(sequence_dict[key][i1][i2][2])
            rsrp_raws = np.array(sequence_dict[key][i1][i2][4])
            rsrq_raws = np.array(sequence_dict[key][i1][i2][5])
            rsrp_list.append(rsrp)
            rsrq_list.append(rsrq)
            rsrp_raws_list.append(rsrp_raws)
            rsrq_raws_list.append(rsrq_raws)
            labels.append(label)
            sequence_dict[key][i1][i2][0] = req_idx
            sequence_dict[key][i1][i2][1] = req_idx
            sequence_dict[key][i1][i2][4] = req_idx
            sequence_dict[key][i1][i2][5] = req_idx
            # sequence_dict[key][i1][i2][2] = req_idx

  rsrp_tts = scaler.fit_transform(rsrp_list)
  rsrq_tts = scaler.fit_transform(rsrq_list)
  rsrp_raws_tts = scaler.fit_transform(rsrp_raws_list)
  rsrq_raws_tts = scaler.fit_transform(rsrq_raws_list)

  # Combining features
  features = np.stack([rsrp_tts, rsrq_tts, rsrp_raws_tts, rsrq_raws_tts], axis=2)

  # Reshaping label data
  labels = np.reshape(labels, (len(labels),1))

  return features, labels, sequence_dict, sequence_dict_orig


## Padding in feature vector generation

### Padding Helper Function definitions

In [6]:
# helper function that checks whether or not the current df will require padding
def padding_condition_check(df, window_size, handover_ts):
  # fails basic length check
  if len(df) < window_size:
    return True
  # calculating time diff
  window1 = df[0:window_size]
  last_entry = df.iloc[window_size-1]
  last_entry_ts = last_entry["timestamp"]
  le_ts_datetime = string_to_datetime(last_entry_ts)
  ho_diff = handover_ts - le_ts_datetime
  ho_diff_seconds = ho_diff.total_seconds()
  # fails time check
  if ho_diff_seconds <= 0.1:
    return True
  return False

# helper function to extract entries from a dataframe and append them to the "entries" list
def extract_entries(df, window_size, ho_ts_datetime):
  entries = []
  for i in range(df.shape[0] - window_size + 1):
    df_slice = df.iloc[i:i+window_size]
    # Extracting the last timestamp for this window
    last_window_entry = df.iloc[i+window_size-1]
    last_window_ts = last_window_entry["timestamp"]
    lw_ts_dt = string_to_datetime(last_window_ts) # last window timestamp datetime object

    # Calculating time to handover and adding it to ttp
    tth = ho_ts_datetime - lw_ts_dt # time to handover
    tth_seconds = tth.total_seconds()

    rsrp = df_slice["rsrp"]
    rsrq = df_slice["rsrq"]
    rsrp_raw = df_slice["rsrp_raw"]
    rsrq_raw = df_slice["rsrq_raw"]
    label = df.iloc[i]
    label = label["label"]

    entry = []
    entry.append(rsrp)
    entry.append(rsrq)
    entry.append(label)
    entry.append(tth_seconds)
    entry.append(rsrp_raw)
    entry.append(rsrq_raw)

    entries.append(entry)

  return entries

# helper function that returns the entries list as it would have appeared in regular execution
def regular_extension(df, window_size):
  entries = []
  # Getting timestamp for handover (or at least as close to it as possible)
  last_entry = df.iloc[-1]
  last_entry_ts = last_entry["timestamp"]
  ho_ts_datetime = string_to_datetime(last_entry_ts)
  if df.shape[0] >= window_size:
    for i in range(0, df.shape[0]-window_size):
      # Extracting the last timestamp for this window
      last_window_entry = df.iloc[i+window_size]
      last_window_ts = last_window_entry["timestamp"]
      lw_ts_dt = string_to_datetime(last_window_ts) # last window timestamp datetime object

      # Calculating time to handover and adding it to ttp
      tth = ho_ts_datetime - lw_ts_dt # time to handover
      tth_seconds = tth.total_seconds()

      # Extracting the rest
      temp = df.iloc[i:i+window_size]
      rsrp = temp["rsrp"]
      rsrq = temp["rsrq"]
      rsrp_raw = temp["rsrp_raw"]
      rsrq_raw = temp["rsrq_raw"]
      label = df.iloc[i]
      label = label["label"]

      entry = []
      entry.append(rsrp)
      entry.append(rsrq)
      entry.append(label)
      entry.append(tth_seconds)
      entry.append(rsrp_raw)
      entry.append(rsrq_raw)

      entries.append(entry)

  return entries

# dataframe containing features is passed as argument to this function along with the window size
# helper function that returns a list of entries for the input dataframe by performing backward padding: i.e., appending the last value to the end as many times as needed
def backward_padding(df, window_size):
  entries = []
  last_entry = df.iloc[-1]
  last_entry_ts = last_entry["timestamp"]
  ho_ts_datetime = string_to_datetime(last_entry_ts)

  # simple padding in case the window is simply not long enough
  if len(df) < window_size:
    for i in range(len(df)):
      # index to check => len(df)-1-i
      df_temp = df[0:i+1]
      num_reps = window_size - 1 - i # number of times the entry needs to be repeated
      to_repeat = df.iloc[i]
      for i in range(num_reps):
        df_temp = pd.concat([df_temp, to_repeat.to_frame().T], ignore_index = True)
      entries_curr_iter = extract_entries(df_temp, window_size, ho_ts_datetime) # entries from the current iteration
      entries.extend(entries_curr_iter)

  # timer check
  else:
    for i in range(df.shape[0]-1):
      # this check ensure that we have all the measurements we can get till this point
      # hence, this iterable should not go past window_size
      if i == window_size:
        break

      df_temp = df[0:i+1]
      num_reps = window_size - 1 - i # number of times the entry needs to be repeated
      to_repeat = df.iloc[i]
      for i in range(num_reps):
        df_temp = pd.concat([df_temp, to_repeat.to_frame().T], ignore_index = True)
      entries_curr_iter = extract_entries(df_temp, window_size, ho_ts_datetime)
      entries.extend(entries_curr_iter)

    # also need regular extension
    reg_extension_entries = regular_extension(df, window_size)
    entries.extend(reg_extension_entries)

  return entries

# extracts "entries" from a file, which is a list of lists containing feature vectors
def add_to_entries(filepath, window_size):
  df = pd.read_csv(filepath)
  # df.drop(["Unnamed: 0"], axis=1, inplace=True)

  ho_entry = df.iloc[-1]
  ho_entry_ts = ho_entry["timestamp"]
  ho_ts_datetime = string_to_datetime(ho_entry_ts)

  entries = []

  if padding_condition_check(df, window_size, ho_ts_datetime):
    # make longer vectors using padding
    # COMMENT THIS LINE OUT TO USE WHICHEVER PADDING METHOD YOU WANT TO TEST
    entries = backward_padding(df, window_size)
    # entries = le_padding(df, window_size)
    return entries
  else:
    # normal execution (without padding)
    entries = regular_extension(df, window_size)
    return entries

### Linear Extrapolation padding implementation

In [15]:
from scipy import interpolate
input_arr = np.array([1.5,2.5])
len_input_arr = len(input_arr)
x_arr = np.arange(0, len(input_arr))
f = interpolate.interp1d(x_arr, input_arr, fill_value='extrapolate')
for i in range(len_input_arr, 6):
    input_arr = np.append(input_arr, f(i))
print(input_arr)

[1.5 2.5 3.5 4.5 5.5 6.5]


In [7]:
# helper function to perform linear extrapolation on a list
def le_meas_list(input, window_size):
    input_arr = np.array(input)
    input_arr_len = len(input_arr) # we save this here to use later as an iterator
    if input_arr_len == 1:
        input_arr = np.repeat(input_arr[0], window_size)
    else:
        x_arr = np.arange(0, input_arr_len)
        f = interpolate.interp1d(x_arr, input_arr, fill_value='extrapolate')
        for i in range(input_arr_len, window_size):
            input_arr = np.append(input_arr, f(i))
    return input_arr

# helper function that performs linear extrapolation padding to a dataframe of entries
def le_df_padder(df, window_size):
    final_df = pd.DataFrame()
    timestamps = list(df["timestamp"])
    for i in range(window_size - len(df)):
        timestamps.append(timestamps[-1])
    final_df["timestamp"] = timestamps
    final_df["rsrp"] = le_meas_list(list(df["rsrp"]), window_size)
    final_df["rsrq"] = le_meas_list(list(df["rsrq"]), window_size)
    final_df["rsrp_raw"] = le_meas_list(list(df["rsrp_raw"]), window_size)
    final_df["rsrq_raw"] = le_meas_list(list(df["rsrq_raw"]), window_size)
    final_df["label"] = [df.iloc[0]["label"] for i in range(window_size)]

    return final_df

## helper function that performs linear extrapolation padding, i.e. extrapolating the values based on a linear gradient so far
def le_padding(df, window_size):
    entries = []
    last_entry = df.iloc[-1]
    last_entry_ts = last_entry["timestamp"]
    ho_ts_datetime = string_to_datetime(last_entry_ts)

    # if not long enough
    if len(df) < window_size:
        for i in range(len(df)):
            temp_df = df.iloc[0:i+1]
            temp_df = le_df_padder(temp_df, window_size)
            entries_curr_iter = extract_entries(temp_df, window_size, ho_ts_datetime)
            entries.extend(entries_curr_iter)

    # if problem is time
    else:
        for i in range(df.shape[0]-1):
            # this check ensure that we have all the measurements we can get till this point
            # hence, this iterable should not go past window_size
            if i == window_size:
                break
            temp_df = df[0:i+1]
            temp_df = le_df_padder(temp_df, window_size)
            entries_curr_iter = extract_entries(temp_df, window_size, ho_ts_datetime)
            entries.extend(entries_curr_iter)
            
        # also need regular extension
        reg_extension_entries = regular_extension(df, window_size)
        entries.extend(reg_extension_entries)

    return entries

### Padding Main Loader

In [7]:
# rsrp_list = []
# rsrq_list = []
# rsrp_raws_list = []
# rsrq_raws_list = []
# labels = []

# Helper function for loading up dataset:
def load_dataset_padding(datasets_list, WINDOW_SIZE):
  # HSR_datasets is a placeholder variable.
  sequence_dict = {}

  diff_count = 0

  ho_event_no = 0 # This counter keeps track of which handover event we have reached
  for master_f in datasets_list:
    dataset_folders = os.listdir(master_f)
    for folder in dataset_folders:
      folder_path = master_f + "/" + folder
      files = os.listdir(folder_path)
      ho_event_no += 1
      curr_hoevent_entries = [] # list containing all the data for the current handover event
      for f in files:
        filepath = folder_path + "/" + f
        len_entries = 0
        len_entries2 = 0
        entries = add_to_entries(filepath, WINDOW_SIZE)
        curr_hoevent_entries.append(entries)
      sequence_dict[ho_event_no] = curr_hoevent_entries

  sequence_dict_orig = sequence_dict.copy()

  print("DIFF COUNT CHECK:", diff_count)

  for key in sequence_dict.keys():
    ho_event_list = sequence_dict[key]
    for i1, entry in enumerate(ho_event_list):
      if entry is not None:
        for i2, subentry in enumerate(entry):
          if subentry[2] == "not_target":
            sequence_dict[key][i1][i2][2] = 0
          else:
            sequence_dict[key][i1][i2][2] = 1

  sequence_dict_renamed = sequence_dict.copy()

  sequence_dict = sequence_dict_orig.copy()

  # Using minmax scaling to bring our features between 0 and 1
  scaler = MinMaxScaler(feature_range=(0,1))

  rsrp_list = []
  rsrq_list = []
  rsrp_raws_list = []
  rsrq_raws_list = []
  labels = []

  for key in sequence_dict.keys():
    ho_event_list = sequence_dict[key]
    if check_valid(sequence_dict, key):
      for i1, entry in enumerate(ho_event_list):
        if entry is not None:
          for i2, subentry in enumerate(entry):
            req_idx = len(rsrp_list)
            rsrp = np.array(sequence_dict[key][i1][i2][0])
            rsrq = np.array(sequence_dict[key][i1][i2][1])
            label = np.array(sequence_dict[key][i1][i2][2])
            rsrp_raws = np.array(sequence_dict[key][i1][i2][4])
            rsrq_raws = np.array(sequence_dict[key][i1][i2][5])
            rsrp_list.append(rsrp)
            rsrq_list.append(rsrq)
            rsrp_raws_list.append(rsrp_raws)
            rsrq_raws_list.append(rsrq_raws)
            labels.append(label)
            sequence_dict[key][i1][i2][0] = req_idx
            sequence_dict[key][i1][i2][1] = req_idx
            sequence_dict[key][i1][i2][4] = req_idx
            sequence_dict[key][i1][i2][5] = req_idx
            # sequence_dict[key][i1][i2][2] = req_idx

  # print(rsrp_list)

  rsrp_tts = scaler.fit_transform(rsrp_list)
  rsrq_tts = scaler.fit_transform(rsrq_list)
  rsrp_raws_tts = scaler.fit_transform(rsrp_raws_list)
  rsrq_raws_tts = scaler.fit_transform(rsrq_raws_list)

  # Combining features
  HSR_features = np.stack([rsrp_tts, rsrq_tts, rsrp_raws_tts, rsrq_raws_tts], axis=2)

  # Reshaping label data
  HSR_labels = np.reshape(labels, (len(labels),1))

  return HSR_features, HSR_labels, sequence_dict, sequence_dict_orig

# BASE STATION PREDICTION

## Evaluating for Top N Base Stations predicted

## Setting up helper functions

In [9]:
# streamlining pipeline by encompassing in functions

# helper function that predicts handoever events, and returns a dictionary which maps each handover event to the time at which the handover event is predicted
def predict_HO_events(input_features, input_labels, model_eval, sequence_dict, sequence_dict_orig, sequence_dict_checker):
  # TODO: add exact variable type information for all the parameters
  # input_features : list of all input features for which predictions have to be made
  # input_labels : list of labels
  # model_eval : keras model being used for evaluation
  # sequence_dict : dictionary keeping track of a measurement and result pairing. i.e., which index in the resulting matrix refers to a specific measurement

  predictions = model_eval.predict(input_features)
  bool_predictions = (predictions > 0.37)
  cm = confusion_matrix(input_labels, bool_predictions)
  tn, fp, fn, tp = cm.ravel()

  earliest_HO_times = {}
  TOTAL_VALID_EVENTS = 0
  FAILED_PREDICTING = 0

  # Predicting handover events
  for HO_COUNT, key in enumerate(sequence_dict.keys()):
    # variable that checks if the handover for the current event has been predicted
    HO_PREDICTED = False
    ho_event_list = sequence_dict[key]
    if check_valid_2(sequence_dict_checker, key):
      TOTAL_VALID_EVENTS += 1
      for i1, entry in enumerate(ho_event_list): # check for the current HO event
        if entry is not None:
          for i2, subentry in enumerate(entry): # this inner loop checks everything for a single base station
            req_idx = sequence_dict[key][i1][i2][0]
            label = input_labels[req_idx][0]
            curr_ts = sequence_dict[key][i1][i2][3]
            prediction = bool_predictions[req_idx]
            if prediction == 1: # if a handover is predicted
              HO_PREDICTED = True
              if key in earliest_HO_times:
                if curr_ts > earliest_HO_times[key]:
                  earliest_HO_times[key] = curr_ts
              else:
                earliest_HO_times[key] = curr_ts
      if HO_PREDICTED == False:
        FAILED_PREDICTING += 1

  return earliest_HO_times, TOTAL_VALID_EVENTS, FAILED_PREDICTING, predictions, bool_predictions

# helper function that gets confidence values/probabilities
def get_probabilities(sequence_dict, sequence_dict_orig, sequence_dict_checker, actual_labels, predictions_probs, predictions_bools, earliest_HO_times):
  HO_to_ts_dict = {}
  HO_label_dict = {}

  # First iteration should be about going through each base station and getting confidence values for each BS
  for HO_COUNT, key in enumerate(sequence_dict.keys()): # Each iteration refers to each Handover event
    ho_event_list = sequence_dict[key]
    HO_to_ts_dict[key] = {}
    if check_valid_2(sequence_dict_checker, key): # Check if this HO event is usable (is the data entry clean enough to be usable?)
      for i1, entry in enumerate(ho_event_list): # i1 can be used to identify the base station
        if entry is not None:
          HO_to_ts_dict[key][i1] = {}
          for i2, subentry in enumerate(entry):
            req_idx = sequence_dict[key][i1][i2][0]
            label = actual_labels[req_idx][0]
            curr_ts = sequence_dict[key][i1][i2][3]
            prediction = predictions_bools[req_idx]
            prediction_probability = predictions_probs[req_idx][0]
            if label == 1:
              HO_label_dict[key] = i1
            try:
              if curr_ts <= earliest_HO_times[key]:
                # handover prediction has happened, base station prediction can be performed
                HO_to_ts_dict[key][i1][curr_ts] = prediction_probability
            except:
              # Evaluated the number of events for which HO was not predicted - add them to our evaluation
              pass

  # HO_to_ts_dict has the following structure:
  # {handover_id : {base_station_id : {timestamp : probability}}}
  # Number of base stations on which state will be replicated. For every handover event, we will consider the top candidates from these two base stations
  earliest_time_BSP = [] # Number of BSP failures can be calculated as a difference in number of handover events predictions, and number of BSP earliest time predictions

  # need to keep track of another pairing:
  # {handover_id : {timestamp : {probability : base_station_id}}} => ts_prob_pairing
  ts_prob_pairing = {}
  # also need to keep a track of a list of sorted probabilities at each timestamp
  # {handover_id: {timestamp : [sorted probabilities] }}
  ts_probabilities = {}

  for handover_key in HO_to_ts_dict.keys():
    all_bs_info = HO_to_ts_dict[handover_key] # for a handover event with id = handover_key, this dict contains information of all BSes
    ts_prob_pairing[handover_key] = {}
    ts_probabilities[handover_key] = {}
    for bs_id in all_bs_info.keys():
      bs_info = all_bs_info[bs_id] # should contain information for a single base station for a single handover event
      for ts in bs_info.keys():
        curr_bs_ts_prob = bs_info[ts]
        # Add pairing to keep track of which probability we are considering
        if ts not in ts_prob_pairing[handover_key].keys():
          ts_prob_pairing[handover_key][ts] = {}
          ts_prob_pairing[handover_key][ts][curr_bs_ts_prob] = bs_id
        else:
          ts_prob_pairing[handover_key][ts][curr_bs_ts_prob] = bs_id
        # Append to list of probabilities
        if ts not in ts_probabilities[handover_key].keys():
          ts_probabilities[handover_key][ts] = [curr_bs_ts_prob]
        else:
          ts_probabilities[handover_key][ts].append(curr_bs_ts_prob)
          ts_probabilities[handover_key][ts] = sorted(ts_probabilities[handover_key][ts], reverse=True)

  return ts_probabilities, ts_prob_pairing, HO_label_dict

# helper function that returns a list with time to prediction
def get_time_to_predictions(ts_probabilities, ts_prob_pairing, HO_label_dict, top_N):
  # ts_probabilities => {handover_id: {timestamp : [sorted probabilities] }}
  # ts_prob_pairing => {handover_id : {timestamp : {probability : base_station_id}}} => ts_prob_pairing
  # HO_label_dict is a dictionary that contains the label index for each handover event
  # top_N is a number which indicates how many BSes we intend to have as targets

  HO_BSP_FAILURES = 0
  earliest_time_BSP = []

  for handover_key in ts_probabilities.keys():
    timestamps = ts_probabilities[handover_key]
    timestamps = sorted(timestamps, reverse=True)
    if timestamps == []:
      continue
    BSP_PREDICTED = False
    actual_label = HO_label_dict[handover_key]
    for ts in timestamps:
      probs = sorted(ts_probabilities[handover_key][ts], reverse=True)
      if BSP_PREDICTED == True:
        break
      num_probs_to_check = min(top_N, len(ts_probabilities[handover_key][ts])) # the number of probabilities to read: this is the min of number of available BSes,
      for i in range(num_probs_to_check):
        if BSP_PREDICTED == True:
          break
        predicted_label = ts_prob_pairing[handover_key][ts][probs[i]]
        if predicted_label == actual_label:
          BSP_PREDICTED = True
          earliest_time_BSP.append(ts)

  return earliest_time_BSP

# helper function to print out the graph
def print_results(earliest_times, output_fname, handover_count):
  # %matplotlib inline

  # Declare num of datapoints
  # N = len(earliest_times)
  N = handover_count
  print("Handover Count:", N)

  x = np.sort(earliest_times)
  y = np.arange(N) / float(N)

  # print(earliest_times)

  # plt.xlabel("Earliest Time to Prediction (seconds)")
  # plt.ylabel("Probability (CDF)")
  # plt.title("CDF for Earliest Time to Prediction (Window Size = 6, Dataset=HSR)")

  # plt.plot(x,y, marker='o')
  # plt.show()
  # print(earliest_times)

  earliest_times_df = pd.DataFrame(earliest_times)
  print(earliest_times_df.describe())

  g100 = 0
  g200 = 0
  g500 = 0
  g1000 = 0
  g90 = 0

  for x in earliest_times:
    if x >= 0.09:
      g90 += 1
    if x >= 0.1:
      g100 += 1
    if x >= 0.2:
      g200 += 1
    if x >= 0.5:
      g500 += 1
    if x >= 1:
      g1000 += 1

  print("100: ", g100/N * 100)
  print("200: ", g200/N * 100)
  print("500: ", g500/N * 100)
  print("1000: ", g1000/N * 100)

  with open(output_fname, "w") as output:
      for item in earliest_times:
          output.write(str(item) + "\n")


# helper function to print out the graph
def print_results_incp(earliest_times, output_fname, handover_count):
  # %matplotlib inline

  # Declare num of datapoints
  N = len(earliest_times)
  # N = handover_count

  # x = np.sort(earliest_times)
  # y = np.arange(N) / float(N)

  # plt.xlabel("Earliest Time to Prediction (seconds)")
  # plt.ylabel("Probability (CDF)")
  # plt.title("CDF for Earliest Time to Prediction (Window Size = 6, Dataset=INCP)")

  # plt.plot(x,y, marker='o')
  # plt.show()

  earliest_times_df = pd.DataFrame(earliest_times)
  print(earliest_times_df.describe())

  g100 = 0
  g200 = 0
  g500 = 0
  g1000 = 0
  g90 = 0

  for x in earliest_times:
    if x >= 0.08:
      g90 += 1
    if x >= 0.1:
      g100 += 1
    if x >= 0.2:
      g200 += 1
    if x >= 0.5:
      g500 += 1
    if x >= 1:
      g1000 += 1
  print("100: ", g100/N * 100)
  print("200: ", g200/N * 100)
  print("500: ", g500/N * 100)
  print("1000: ", g1000/N * 100)

  with open(output_fname, "w") as output:
      for item in earliest_times:
          output.write(str(item) + "\n")

In [10]:
# putting all the above helper functions together
def get_top_n_results(model_eval, input_features, input_labels, sequence_dict, sequence_dict_orig, sequence_dict_checker, top_N, output_fname):
  # predicting handover events and getting relevant timestamps
  earliest_HO_times, TOTAL_VALID_EVENTS, FAILED_PREDICTING, predictions_probs, predictions_bools = predict_HO_events(input_features, input_labels, model_eval, sequence_dict, sequence_dict_orig, sequence_dict_checker)
  # getting relevant probability relations and pairings with handover events
  ts_probabilities, ts_prob_pairing, HO_label_dict = get_probabilities(sequence_dict, sequence_dict_orig, sequence_dict_checker, input_labels, predictions_probs, predictions_bools, earliest_HO_times)
  # getting a list of earliest time to predictions
  earliest_times_BSP = get_time_to_predictions(ts_probabilities, ts_prob_pairing, HO_label_dict, top_N)
  # generating graphs and getting the relevant results
  print_results(earliest_times_BSP, output_fname, TOTAL_VALID_EVENTS)

# putting all the above helper functions together
def get_top_n_results_incp(model_eval, input_features, input_labels, sequence_dict, sequence_dict_orig, sequence_dict_checker, top_N, output_fname):
  # predicting handover events and getting relevant timestamps
  earliest_HO_times, TOTAL_VALID_EVENTS, FAILED_PREDICTING, predictions_probs, predictions_bools = predict_HO_events(input_features, input_labels, model_eval, sequence_dict, sequence_dict_orig, sequence_dict_checker)
  # getting relevant probability relations and pairings with handover events
  ts_probabilities, ts_prob_pairing, HO_label_dict = get_probabilities(sequence_dict, sequence_dict_orig, sequence_dict_checker, input_labels, predictions_probs, predictions_bools, earliest_HO_times)
  # getting a list of earliest time to predictions
  earliest_times_BSP = get_time_to_predictions(ts_probabilities, ts_prob_pairing, HO_label_dict, top_N)
  # generating graphs and getting the relevant results
  print_results_incp(earliest_times_BSP, output_fname, TOTAL_VALID_EVENTS)


## Evaluating a theoretical maximum

In [21]:
def max_achievable(sequence_dict, sequence_dict_checker, actual_labels):
    best_possible = []

    # extracting the best possible times from the loaded data
    for HO_COUNT, key in enumerate(sequence_dict.keys()): # Each iteration refers to each Handover event
        ho_event_list = sequence_dict[key]
        if check_valid_2(sequence_dict_checker, key): # Check if this HO event is usable (is the data entry clean enough to be usable?)
            time_noted = False
            for i1, entry in enumerate(ho_event_list): # i1 can be used to identify the base station
                    if entry is not None:
                        if not time_noted:
                            for i2, subentry in enumerate(entry):
                                req_idx = sequence_dict[key][i1][i2][0]
                                label = actual_labels[req_idx][0]
                                curr_ts = sequence_dict[key][i1][i2][3]
                                if label:
                                    best_possible.append(curr_ts)
                                    time_noted = True
                                    break
                        else:
                            break
    
    # Evaluating the best possible times
    df = pd.DataFrame(best_possible)
    print(df.describe())

    N = len(best_possible)
    g100 = 0
    g200 = 0
    g500 = 0
    g1000 = 0

    for x in best_possible:
        if x >= 0.1:
            g100 += 1
        if x >= 0.2:
            g200 += 1
        if x >= 0.5:
            g500 += 1
        if x >= 1:
            g1000 += 1

    print("100: ", g100/N * 100)
    print("200: ", g200/N * 100)
    print("500: ", g500/N * 100)
    print("1000: ", g1000/N * 100)

                            
# max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)


## Getting the HSR Dataset

Get the dataset from the following link:
https://anonymous.4open.science/api/repo/EdgeCatBSPDataset-EDED/file/hsr_extended_dataset.zip

Unzip the dataset after downloading.


You may need to adjust the paths in the "HSR_datasets" field with respect to where you have saved the dataset.


## Evaluation of HSR Dataset

### HSR DATASET PART 0

In [22]:
HSR_datasets = ["HSR_Extended_4_0"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)

DIFF COUNT CHECK: 0
                 0
count  3319.000000
mean      1.837007
std       0.520614
min       0.000000
25%       1.716181
50%       2.119993
75%       2.160004
max       2.402126
100:  99.57818620066286
200:  98.88520638746611
500:  95.26965953600482
1000:  90.05724615848148


In [13]:
print(HSR_features.shape)

(336299, 6, 4)


In [14]:
for i in range(1,6):
  output_fname = "HSR_0" + str(i) + "_earliest.txt"
  get_top_n_results(model_eval, HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig, HSR_seq_dict_checker, i, output_fname)

2023-06-26 23:15:58.693180: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-26 23:15:58.702098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-26 23:15:58.706200: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

10510/10510 [==============================] - 76s 7ms/step
Handover Count: 3319
                 0
count  3283.000000
mean      1.668875
std       0.591842
min       0.037904
25%       1.315791
50%       1.920797
75%       2.158667
max       2.402126
100:  98.3127448026514
200:  97.49924676107261
500:  92.43748116902681
1000:  82.13317264236217
10510/10510 [==============================] - 69s 7ms/step
Handover Count: 3319
                 0
count  3302.000000
mean      1.665245
std       0.597766
min       0.000000
25%       1.308395
50%       1.921101
75%       2.158673
max       2.402126
100:  98.583910816511
200:  97.77041277493221
500:  92.6785176257909
1000:  82.46459777041278
10510/10510 [==============================] - 67s 6ms/step
Handover Count: 3319
                 0
count  3305.000000
mean      1.664281
std       0.598938
min       0.000000
25%       1.300038
50%       1.920041
75%       2.158673
max       2.402126
100:  98.6140403736065
200:  97.80054233202772
500:  9

### HSR DATASET PART 1

In [15]:
HSR_datasets = ["HSR_Extended_4_1"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

for i in range(1,6):
  output_fname = "HSR_1" + str(i) + "_earliest.txt"
  get_top_n_results(model_eval, HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig, HSR_seq_dict_checker, i, output_fname)

DIFF COUNT CHECK: 0
10587/10587 [==============================] - 64s 6ms/step
Handover Count: 3367
                 0
count  3330.000000
mean      1.693197
std       0.580484
min       0.000000
25%       1.322658
50%       1.960011
75%       2.158555
max       2.402126
100:  98.42589842589842
200:  97.2081972081972
500:  92.96109296109296
1000:  83.6055836055836
10587/10587 [==============================] - 62s 6ms/step
Handover Count: 3367
                 0
count  3351.000000
mean      1.689706
std       0.585409
min       0.000000
25%       1.321960
50%       1.959989
75%       2.158420
max       2.402126
100:  98.72289872289872
200:  97.5051975051975
500:  93.34719334719335
1000:  84.08078408078407
10587/10587 [==============================] - 62s 6ms/step
Handover Count: 3367
                 0
count  3353.000000
mean      1.689523
std       0.585765
min       0.000000
25%       1.322165
50%       1.959989
75%       2.158415
max       2.402126
100:  98.72289872289872
200:  97.

In [23]:
HSR_datasets = ["HSR_Extended_4_1"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)

DIFF COUNT CHECK: 0
                 0
count  3367.000000
mean      1.840130
std       0.519271
min       0.000000
25%       1.722562
50%       2.120002
75%       2.160002
max       2.402126
100:  99.67329967329968
200:  98.6040986040986
500:  95.27769527769527
1000:  90.08019008019008


### HSR DATASET PART 2

In [16]:
HSR_datasets = ["HSR_Extended_4_2"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

for i in range(1,6):
  output_fname = "HSR_2" + str(i) + "_earliest.txt"
  get_top_n_results(model_eval, HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig, HSR_seq_dict_checker, i, output_fname)

DIFF COUNT CHECK: 0
10530/10530 [==============================] - 62s 6ms/step
Handover Count: 3327
                 0
count  3282.000000
mean      1.647124
std       0.602567
min       0.000000
25%       1.214149
50%       1.915754
75%       2.158024
max       2.207901
100:  97.74571686203787
200:  96.99428914938383
500:  91.64412383528705
1000:  80.64322212203187
10530/10530 [==============================] - 62s 6ms/step
Handover Count: 3327
                 0
count  3303.000000
mean      1.642268
std       0.609611
min       0.000000
25%       1.206427
50%       1.913940
75%       2.157989
max       2.207901
100:  98.04628794709949
200:  97.23474601743312
500:  91.85452359483017
1000:  80.94379320709348
10530/10530 [==============================] - 62s 6ms/step
Handover Count: 3327
                 0
count  3310.000000
mean      1.639280
std       0.613292
min       0.000000
25%       1.200368
50%       1.913799
75%       2.157975
max       2.207901
100:  98.04628794709949
200:  

In [24]:
HSR_datasets = ["HSR_Extended_4_2"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)

DIFF COUNT CHECK: 0
                 0
count  3327.000000
mean      1.837313
std       0.526419
min       0.000000
25%       1.720002
50%       2.120003
75%       2.160001
max       2.237871
100:  99.42891493838293
200:  98.67748722572888
500:  95.28103396453261
1000:  89.26961226330027


### HSR DATASET PART 3

In [17]:
HSR_datasets = ["HSR_Extended_4_3"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

for i in range(1,6):
  output_fname = "HSR_3" + str(i) + "_earliest.txt"
  get_top_n_results(model_eval, HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig, HSR_seq_dict_checker, i, output_fname)

DIFF COUNT CHECK: 0
10576/10576 [==============================] - 62s 6ms/step
Handover Count: 3359
                 0
count  3314.000000
mean      1.666375
std       0.588720
min       0.000000
25%       1.280015
50%       1.920009
75%       2.158186
max       2.210000
100:  98.09467103304554
200:  97.38017267043763
500:  92.4679964275082
1000:  81.89937481393271
10576/10576 [==============================] - 67s 6ms/step
Handover Count: 3359
                 0
count  3335.000000
mean      1.664908
std       0.592158
min       0.000000
25%       1.280015
50%       1.920001
75%       2.158183
max       2.210000
100:  98.48169097945816
200:  97.73742185174159
500:  92.76570407859482
1000:  82.49479011610599
10576/10576 [==============================] - 62s 6ms/step
Handover Count: 3359
                 0
count  3339.000000
mean      1.663237
std       0.594219
min       0.000000
25%       1.280010
50%       1.919999
75%       2.158176
max       2.210000
100:  98.48169097945816
200:  9

In [25]:
HSR_datasets = ["HSR_Extended_4_3"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)

DIFF COUNT CHECK: 0
                 0
count  3359.000000
mean      1.832966
std       0.521397
min       0.000000
25%       1.693121
50%       2.119296
75%       2.160001
max       2.210000
100:  99.61298005358738
200:  98.92825245608812
500:  95.44507293837452
1000:  89.66954450729384


### HSR DATASET PART 4

In [18]:
HSR_datasets = ["HSR_Extended_4_4"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

for i in range(1,6):
  output_fname = "HSR_4" + str(i) + "_earliest.txt"
  get_top_n_results(model_eval, HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig, HSR_seq_dict_checker, i, output_fname)

DIFF COUNT CHECK: 0
10689/10689 [==============================] - 63s 6ms/step
Handover Count: 3312
                 0
count  3260.000000
mean      1.632724
std       0.600332
min       0.000000
25%       1.199997
50%       1.877832
75%       2.157883
max       2.204813
100:  97.52415458937197
200:  96.46739130434783
500:  92.11956521739131
1000:  80.012077294686
10689/10689 [==============================] - 63s 6ms/step
Handover Count: 3312
                 0
count  3279.000000
mean      1.632893
std       0.603006
min       0.000000
25%       1.200033
50%       1.877886
75%       2.157883
max       2.204813
100:  97.88647342995169
200:  96.82971014492753
500:  92.45169082125604
1000:  80.58574879227052
10689/10689 [==============================] - 63s 6ms/step
Handover Count: 3312
                 0
count  3282.000000
mean      1.631804
std       0.604363
min       0.000000
25%       1.200015
50%       1.877880
75%       2.157882
max       2.204813
100:  97.91666666666666
200:  96

In [26]:
HSR_datasets = ["HSR_Extended_4_4"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)

DIFF COUNT CHECK: 0
                 0
count  3312.000000
mean      1.833156
std       0.528459
min       0.000000
25%       1.717764
50%       2.119992
75%       2.160004
max       2.237871
100:  99.2451690821256
200:  98.52053140096618
500:  95.44082125603865
1000:  89.58333333333334


### HSR FineGrained

In [19]:
HSR_datasets = ["../with_RAW_CNXT/HSR_August_Dataset/", "../with_RAW_CNXT/HSR_July_Dataset/"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

for i in range(1,6):
  output_fname = "HSR_FG" + str(i) + "_earliest.txt"
  get_top_n_results(model_eval, HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig, HSR_seq_dict_checker, i, output_fname)

DIFF COUNT CHECK: 0
507/507 [==============================] - 4s 8ms/step
Handover Count: 405
                0
count  370.000000
mean     0.945824
std      0.530116
min      0.000000
25%      0.485334
50%      0.957867
75%      1.346004
max      2.287975
100:  84.19753086419753
200:  81.48148148148148
500:  67.90123456790124
1000:  44.44444444444444
507/507 [==============================] - 3s 6ms/step
Handover Count: 405
                0
count  396.000000
mean     0.955050
std      0.538690
min      0.000000
25%      0.517519
50%      0.959215
75%      1.360547
max      2.287975
100:  89.62962962962962
200:  86.66666666666667
500:  73.58024691358025
1000:  48.148148148148145
507/507 [==============================] - 3s 7ms/step
Handover Count: 405
                0
count  399.000000
mean     0.977261
std      0.534329
min      0.000000
25%      0.560897
50%      1.000748
75%      1.368553
max      2.287975
100:  90.61728395061729
200:  87.90123456790123
500:  76.29629629629629
10

In [27]:
HSR_datasets = ["../with_RAW_CNXT/HSR_August_Dataset/", "../with_RAW_CNXT/HSR_July_Dataset/"]

_, _, HSR_seq_dict_checker, HSR_seq_dict_checker_orig = load_dataset(HSR_datasets, 6)
HSR_features, HSR_labels, HSR_seq_dict, HSR_seq_dict_orig = load_dataset_padding(HSR_datasets, 6)

max_achievable(HSR_seq_dict, HSR_seq_dict_checker, HSR_labels)

DIFF COUNT CHECK: 0
                0
count  405.000000
mean     1.009009
std      0.543597
min      0.000000
25%      0.562871
50%      1.073573
75%      1.401144
max      2.287975
100:  92.8395061728395
200:  90.12345679012346
500:  77.53086419753087
1000:  54.32098765432099
